# To do

2. write a function to get the roster of a given team in a given season 
   - 2a Write a function to get individual team pers and games palyers of team TOT 

3. partition data for predictions:
    - what are we considering? we can make a model that has an input for:
        - x number of historical head to head matchups  
        - team roster pers 
        - injury data 
        - regular season info 
    
        
4. Decide test train partitions:
    - previous 2 seasons for next season based on literature 
    
4. test model architerctures 

5. Decide on best model architecture 

5. test Hyerparameters 

6. Report 

<!-- further work  -->
<!-- per predictor based on college stats found on each players basketball reference  -->

In [1]:
ls()


zsh: parse error near `()'


In [2]:
import pandas as pd 
import numpy as np 
from os import listdir as ls
import tensorflow as tf
from sklearn.decomposition import PCA

In [3]:
v1 = pd.read_parquet('[2015, 2016]_game_stats.parquet.gzip')
v2 = pd.read_parquet('[2017, 2018]_game_stats.parquet.gzip')
v3 = pd.read_parquet('[2019, 2020]_game_stats.parquet.gzip')
v4 = pd.read_parquet('[2021, 2022]_game_stats.parquet.gzip')

season_reg = pd.concat([v1, v2, v3, v4], axis = 0)
season_reg.to_csv("season_reg.csv")

In [4]:
def select_season(season_string):
    seasons = pd.read_csv("season_reg.csv")
    seasons['date'] = pd.to_datetime(seasons['date'])
    start_date = f'20{season_string[2:4]}-10-03'
    end_date = f'20{season_string[-2:]}-04-30'
    return seasons[(seasons['date'] >= start_date) & (seasons['date'] <= end_date)].drop('Unnamed: 0', axis = 1).reset_index

# Player Pers

In [5]:
players_teams=pd.read_csv('seasons_per_s.csv')
print(players_teams['team_id'].unique(), len(players_teams['team_id'].unique()))
# what is TOT

['GSW' 'PHO' 'DAL' 'TOT' 'SAC' 'CLE' 'LAL' 'ATL' 'CHI' 'NYK' 'MEM' 'IND'
 'MIA' 'BOS' 'SAS' 'PHI' 'MIN' 'LAC' 'ORL' 'DET' 'BRK' 'MIL' 'NOP' 'CHO'
 'HOU' 'WAS' 'DEN' 'UTA' 'TOR' 'POR' 'OKC'] 31


In [6]:
# TOT means a players played on multiple teams 
players_teams[players_teams['team_id']=='TOT']

,season,team_id,player_dir,per
7,2018-19,TOT,/players/b/barneha02.html,12.8
15,2016-17,TOT,/players/b/bogutan01.html,9.3
28,2015-16,TOT,/players/h/holidju01.html,9.4
33,2018-19,TOT,/players/h/holidju01.html,10.0
38,2021-22,TOT,/players/h/holidju01.html,9.5
...,...,...,...,...
5036,2021-22,TOT,/players/k/keybr01.html,13.7
5054,2021-22,TOT,/players/b/brownch05.html,6.4
5058,2021-22,TOT,/players/h/hillma01.html,12.7
5077,2021-22,TOT,/players/j/johnske07.html,8.7


In [7]:
barneha02 = players_teams[players_teams['player_dir']=='/players/b/barneha02.html'] 
barneha02[barneha02['season']=='2018-19'] 

,season,team_id,player_dir,per
7,2018-19,TOT,/players/b/barneha02.html,12.8
8,2018-19,DAL,/players/b/barneha02.html,13.1
9,2018-19,SAC,/players/b/barneha02.html,12.3


In [8]:
#since we have individual team stats WE NEED TO FIGURE OUT WHO PLAYED WHERE AND WHEN 
multi_team_players = players_teams[players_teams['team_id']=='TOT'] 
multi_team_players

,season,team_id,player_dir,per
7,2018-19,TOT,/players/b/barneha02.html,12.8
15,2016-17,TOT,/players/b/bogutan01.html,9.3
28,2015-16,TOT,/players/h/holidju01.html,9.4
33,2018-19,TOT,/players/h/holidju01.html,10.0
38,2021-22,TOT,/players/h/holidju01.html,9.5
...,...,...,...,...
5036,2021-22,TOT,/players/k/keybr01.html,13.7
5054,2021-22,TOT,/players/b/brownch05.html,6.4
5058,2021-22,TOT,/players/h/hillma01.html,12.7
5077,2021-22,TOT,/players/j/johnske07.html,8.7


In [9]:
# 2a. Write a function to get individual team pers and games palyers of team TOT 

# need to scrape from tathead .com using the players dir from player dir 
    # ex. /players/b/barneha02.html -> https://stathead.com/basketball/player-game-finder.cgi?player_id=barneha02
    
#then use updated per data to add dates players were on a team.

In [10]:
cleaned_reg_season = season_reg.copy()
columns = season_reg.columns
date_features = ['date']
string_features = [ 'home_team', 'away_team', 'arena']
numeric_features = [ c for c in columns if c not in string_features + date_features]

cleaned_reg_season[numeric_features] = cleaned_reg_season[numeric_features].apply(pd.to_numeric)
cleaned_reg_season[date_features] = cleaned_reg_season[date_features].apply(pd.to_datetime)

In [11]:
reg_2021 = cleaned_reg_season[(cleaned_reg_season["date"] >= "2021-10-19") & (cleaned_reg_season["date"] <= "2022-04-10")]
reg_2021 = reg_2021.drop(["date", "home_team", "away_team", "arena"] , axis=1)
reg_2019 = cleaned_reg_season[(cleaned_reg_season["date"] >= "2021-10") & (cleaned_reg_season["date"] <= "2022-04-30")]
reg_2019 = reg_2019.drop(["date", "home_team", "away_team", "arena"] , axis=1)

pca = PCA(n_components=30)
principal_components = pca.fit_transform(reg_2021)
principal_components = pd.DataFrame(principal_components)
principal_components


pca_2019 = pd.DataFrame(pca.transform(reg_2019))

In [12]:
spread_2021 = pd.DataFrame(reg_2021["home_pts"] - reg_2021["away_pts"],  columns= ["spread"])
#spread_2021["home_win"] = 0
#spread_2021.loc[spread_2021["spread"] > 0, "home_win"] = 1

spread_2019 = pd.DataFrame(reg_2019["home_pts"] - reg_2019["away_pts"],  columns= ["spread"])
#spread_2019["home_win"] = 0
#spread_2019.loc[spread_2019["spread"] > 0, "home_win"] = 1

In [13]:
pca_2019.head()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,4.406874,21.262142,14.063243,8.109345,-4.155525,8.926840,-6.592584,9.163101,-0.402483,-7.265980,...,1.057068,0.189488,0.493340,1.473516,-2.514763,0.934779,-0.015580,-0.016320,-0.015869,-0.001889
1,10.736358,-5.466881,13.407105,12.085999,-4.858855,1.161963,4.978612,-6.066800,-0.104653,-2.391912,...,0.526710,3.338775,-1.748323,2.618372,1.719131,1.020514,0.002655,0.049862,0.000461,0.001063
2,20.705473,-0.676365,12.960276,9.159453,3.011470,4.590910,-6.219167,6.442562,17.830488,-2.993369,...,-0.946691,1.207371,-0.195029,2.147250,0.137171,2.288116,0.019896,-0.017673,0.006715,-0.005847
3,-34.125975,-6.218503,4.202907,-1.307543,-2.611409,-9.169314,-10.085612,3.072512,-2.881526,2.071789,...,-2.514117,0.673121,-1.715811,0.116725,-0.056818,-0.140287,-0.021695,-0.005934,-0.007387,0.037068
4,43.362451,4.801790,33.423928,7.935148,24.043041,0.869611,9.723296,-4.164655,6.044178,2.483088,...,-0.243441,0.742674,0.767364,1.555003,2.400053,-0.982017,0.002133,-0.013890,0.009044,0.005970


In [ ]:
# Define input layers for each input
home_per_input = tf.keras.Input(shape=(None, 10), name='home_per_input')
home_team_input = tf.keras.Input(shape=(None, 30), name='home_team_input')
season_pca_input = tf.keras.Input(shape=(None, 30), name='season_pca_input')
arena_input = tf.keras.Input(shape=(None, 51), name='arena_input')
away_team_input = tf.keras.Input(shape=(None, 30), name='away_team_input')
away_per_input = tf.keras.Input(shape=(None, 10), name='away_per_input')

# Concatenate or merge the inputs
concatenated_inputs = tf.keras.layers.concatenate([home_per_input, home_team_input, season_pca_input, arena_input, away_team_input, away_per_input])

# Define the rest of your model architecture
x = tf.keras.layers.Dense(161, activation="relu")(concatenated_inputs)
x = tf.keras.layers.Dense(250, activation="relu")(x)
x = tf.keras.layers.Dense(250, activation="relu")(x)
x = tf.keras.layers.Dense(25, activation="relu")(x)
x = tf.keras.layers.Dense(25, activation="relu")(x)
output = tf.keras.layers.Dense(1, activation="linear")(x)

# Create the model
bigger_model = tf.keras.Model(inputs=[home_per_input, home_team_input, season_pca_input, arena_input, away_team_input, away_per_input], outputs=output)

# Compile the model
bigger_model.compile(loss='mean_squared_error', optimizer="adam", metrics=["accuracy", "mean_squared_error"])

# Print the model summary
bigger_model.summary())

Epoch 1/10


2024-04-03 21:16:15.108889: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-04-03 21:16:15.108931: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-04-03 21:16:15.108947: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-04-03 21:16:15.109300: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-04-03 21:16:15.109684: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


model.evaluate(reg_2021, spread_2021)

In [ ]:
import matplotlib.pyplot as plt

predictions = pd.DataFrame(model.predict(reg_2021).flatten())

predicted_values = predictions
observed_values = spread_2021


# Plot histogram
plt.hist(predicted_values, bins=40, label='Predicted', alpha=0.5)
plt.hist(observed_values, bins=40, label='Observed', alpha=0.5)
plt.xlabel('Values')
plt.ylabel('Frequency')
plt.title('Histogram of Predicted vs Observed Values')
plt.legend()
plt.show()

In [5]:
# +/- 15 spread categorical 

SyntaxError: invalid syntax (2618411577.py, line 1)